In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [2]:
from transformers import (
    GPT2Config,
    GPT2Tokenizer,
    GPT2LMHeadModel,
)

/home/cgagne/cvar_generation/conda_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
modelname = '../models/pretrained/gpt2-large/'
config = GPT2Config.from_pretrained(modelname)
tokenizer = GPT2Tokenizer.from_pretrained(modelname)

In [4]:
import sys
sys.path.append('../')
from rl_learner import TD_Learner
import torch
import pickle
device = 'cpu'
import sys
sys.path.append('../')

from cvar_helpers import calc_cvar_from_samples
from train_rl_batch_sentence_chains import average_states_by_period

### Code

In [12]:
def load_rl_model(epoch=48, hidden_dim =100, n_quantiles = 10, extra = '_composite_rewards',
                  folder = 'sentence_chains_I_4',huber=0.1):

    filename='../data/results/'+folder+'/'
    filename+=f'quantile_learner_{hidden_dim}_{huber}{extra}/log_quantile_learner_epoch{epoch}.pkl'

    Z_network = TD_Learner(config.n_embd, n_quantiles, hidden_dim=hidden_dim)
    Z_network.load_state_dict(torch.load(filename.replace('log_',''),map_location=torch.device('cpu')))

    log = pickle.load(open(filename,'rb'))
    loss = np.array(log['loss'])
    epoch = np.array(log['epoch'])
    
    out = {'Z_network': Z_network,
             'loss': loss}
    return(out)

### Comparing v5 Models

In [13]:
out = load_rl_model(epoch=19, hidden_dim =100, n_quantiles = 10, extra = '_composite_rewards',
                          folder = 'sentence_chains_I_5',huber=0.1)

In [15]:
v5_stems_1 =["Tomorrow will be an easy day.",
           "Tomorrow will be a busy day.",
           "Tomorrow will be a difficult day."]
v5_stems_2 = ["I need to go to the doctor's.", 'I need to go to work.', 
         'I need to go to the grocery store.', 'I need to clean the house.',  'I need to take that test.']

